In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Foo Fighters,What If I Do?,Back and forth that voice of yours keeps me up...
1,Meat Loaf,Rock N Roll Mercenaries,"(Talking, talking, wealth, talking, money, tal..."
2,Alison Krauss,Stars,Stood out in the rain \r\nLet it soak me down...
3,Xavier Rudd,My Missing,I See you sleeping only in my mind \r\nI have...
4,Face To Face,I Used To Think,I've tried to fake it \r\nI've tried to lie a...
5,Frank Sinatra,Five Hundred Guys,Five hundred guys have one thousand eyes for m...
6,Townes Van Zandt,The Hole,The old woman finally caught me \r\nSneakin' ...
7,Kirk Franklin,All I Need,You are all I need \r\nI will sing lift up my...
8,Christmas Songs,Christmas In Dixie,By now in New York City. \r\nThere's snow on ...
9,Steely Dan,Fm,"Worry the bottle Mamma, it's grapefruit wine ..."


In [40]:
df.tail(10)

,artist,song,text
4990,Conway Twitty,As Soon As I Hang Up The Phone,( loretta ) hello ( conway ) hello loretta ( l...
4991,Hank Williams,(I Heard That) Lonesome Whistle,"i wa ride number nine , head south from caroli..."
4992,Kiss,Danger,"in the night i will be waitin ' readi to go , ..."
4993,Emmylou Harris,Didn't Leave Nothing But The Baby,go to sleep you littl babi go to sleep you lit...
4994,Ray Boltz,Does The Light Still Shine,doe the light still shine ? word by ray boltz ...
4995,Neil Sedaka,Is This The Way To Cross The Street?,"sha , la , la , la , la , la , la , la sha , l..."
4996,Cher,Disaster Cake,if you 're gon na make an omelet you got ta br...
4997,Soundgarden,Live To Rise,"what if all you understand , could fit into th..."
4998,Aiza Seguerra,Put A Little Love In Your Heart,think of your fellowmen lend them a help hand ...
4999,Z-Ro,To Love A Thug,"( talk ) look out man , quit trip man , how yo..."


In [44]:
df.tail(0)

,artist,song,text


In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [13]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [14]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [15]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.03383274, 0.04009587, ..., 0.0316723 , 0.09102702,
       0.06017442])

In [24]:
df[df['song'] == 'Good Old Days']

,artist,song,text


In [50]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [52]:
# def recommendation(song_name):
#     # Check if the song exists in the DataFrame
#     if song_name not in df['song'].values:
#         return f"❌ Song '{song_name}' not found in the dataset."

#     idx = df[df['song'] == song_name].index[0]
#     distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

#     songs = []
#     for i in distances[1:6]:
#         songs.append(df.iloc[i[0]]['song'])

#     return songs


In [54]:
recommendation('Danger')

['Dangerous',
 'Danger Us',
 'Beautiful Calm Driving',
 "Steppin' Razor",
 'Dangerous Game',
 "Billy's Got A Gun",
 'Same Direction',
 'I Am Your Child',
 'Friday',
 'Soul Motion',
 'Going Through Changes',
 "I've Got A Funny Feeling",
 'World In Motion',
 'Hit Between The Eyes',
 'Foolish Love',
 'Them There Eyes',
 'Anguish And Fear',
 'Heart Of Hearts',
 'Dangerous',
 "You're The One"]

In [56]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))